# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [12]:
# Group the dataframe by 'CustomerID' and 'ProductID' and sum the 'Quantity' field
group = data.groupby(by=['CustomerID', 'ProductID']).sum()['Quantity'].reset_index()

# Check the result
group

,CustomerID,ProductID,Quantity
0,33,23,1
1,33,51,1
2,33,53,1
3,33,67,1
4,33,70,1
...,...,...,...
63623,98200,392,25
63624,98200,408,25
63625,98200,418,25
63626,98200,422,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [110]:
# Create a pivot table
pt = pd.pivot_table(group, values='Quantity', index='CustomerID', columns='ProductID', fill_value=0)

# Check the result
pt

ProductID,1,2,3,4,5,6,7,8,9,10,...,443,444,445,446,447,448,449,450,451,452
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
200,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
264,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
356,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0
412,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,0,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98069,25,0,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,25,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,0,25,0,0,0,0,25,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [94]:
# Create a customer similarity matrix
sim_matrix = 1 / (1 + pd.DataFrame(squareform(pdist(pt, metric='euclidean')), index=pt.index, columns=pt.index))

# Check the result
sim_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [105]:
sim_matrix.loc[0:1000 , :]

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
464,0.082709,0.075255,0.080350,0.078187,0.078711,1.000000,0.074798,0.084626,0.080070,0.078187,...,0.004816,0.004687,0.004416,0.005038,0.004308,0.004533,0.004578,0.004363,0.004167,0.004347
477,0.074573,0.075956,0.076923,0.075025,0.075025,0.074798,1.000000,0.078977,0.075720,0.074573,...,0.004823,0.004688,0.004426,0.005037,0.004321,0.004534,0.004589,0.004366,0.004168,0.004338
639,0.083020,0.076435,0.080634,0.082403,0.082403,0.084626,0.078977,1.000000,0.085983,0.083020,...,0.004816,0.004682,0.004412,0.005035,0.004308,0.004526,0.004581,0.004363,0.004176,0.004335
649,0.081503,0.077674,0.082100,0.077171,0.078187,0.080070,0.075720,0.085983,1.000000,0.079792,...,0.004810,0.004676,0.004414,0.005035,0.004314,0.004528,0.004585,0.004369,0.004172,0.004329


In [97]:
for index in sim_matrix.index:
    print(index)
    print(sim_matrix.iloc[index, :])

33
CustomerID
33       0.083333
200      0.079246
264      0.088152
356      0.083333
412      0.079792
           ...   
97928    0.004533
98069    0.004593
98159    0.004372
98185    0.004181
98200    0.004335
Name: 2617, Length: 1000, dtype: float64
200
CustomerID
33       0.022745
200      0.022520
264      0.022450
356      0.022462
412      0.022555
           ...   
97928    0.004571
98069    0.004620
98159    0.004328
98185    0.004174
98200    0.004333
Name: 18683, Length: 1000, dtype: float64
264
CustomerID
33       0.016507
200      0.016895
264      0.016735
356      0.016783
412      0.016416
           ...   
97928    0.004505
98069    0.004531
98159    0.004342
98185    0.004176
98200    0.004289
Name: 25209, Length: 1000, dtype: float64
356
CustomerID
33       0.011584
200      0.011599
264      0.011656
356      0.011578
412      0.011763
           ...   
97928    0.004440
98069    0.004340
98159    0.004215
98185    0.004282
98200    0.004225
Name: 33381, Length: 100

IndexError: single positional indexer is out-of-bounds

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.